In [4]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge bs4 --yes

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import geocoder
import requests 
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    bs4-4.9.1                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         223 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.1-0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
bs4-4.9.1 

In [5]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Populated_places_in_Riyadh_Province").text
soup = BeautifulSoup(data, 'html.parser')

In [6]:
neighborhoodList = []
for i in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(i.text)

In [7]:
rdf = pd.DataFrame({"Neighborhood": neighborhoodList})
rdf.shape
rdf.head()

,Neighborhood
0,Afif
1,Al Artawiyah
2,Al Bir
3,Al Zulfi
4,Al-Ghat


In [8]:
#function to get coordinates of neighborhoods

def get_latlng(neighborhood):
    lat_long_coords = None
    while(lat_long_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_long_coords = g.latlng
    return lat_long_coords

In [9]:
coords = [get_latlng(neighborhood) for neighborhood in rdf["Neighborhood"].tolist()]
coords

[[3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.1109200000000214, 101.66187000000008],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.1432700000000295, 101.71070000000009],
 [3.147890000000075, 101.6940500

In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
rdf['Latitude'] = df_coords['Latitude']
rdf['Longitude'] = df_coords['Longitude']

rdf.to_csv("rdf.csv", index=False)

print(rdf.shape)
rdf.head()

(34, 3)


,Neighborhood,Latitude,Longitude
0,Afif,3.14789,101.69405
1,Al Artawiyah,3.14789,101.69405
2,Al Bir,3.14789,101.69405
3,Al Zulfi,3.14789,101.69405
4,Al-Ghat,3.14789,101.69405


In [13]:
address = 'Riyadh, Saudi Arabia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of Riyadh are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Riyadh are 24.6319692, 46.7150648.


In [15]:
mapr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(rdf['Latitude'], rdf['Longitude'], rdf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mapr)  

mapr.save('mapr.html')
mapr

In [18]:
CLIENT_ID = 'ID' #Foursquare ID
CLIENT_SECRET = 'Secret' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(rdf['Latitude'], rdf['Longitude'], rdf['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(3400, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Afif,3.14789,101.69405,Urbanscapes House,3.146803,101.696028,Exhibit
1,Afif,3.14789,101.69405,BackHome Kuala Lumpur,3.148732,101.697887,Hostel
2,Afif,3.14789,101.69405,Adya Hotel Kuala Lumpur,3.151703,101.695623,Hotel
3,Afif,3.14789,101.69405,Shuang Xi 雙喜,3.142853,101.696465,Speakeasy
4,Afif,3.14789,101.69405,Bangunan Sultan Abdul Samad,3.148620,101.694549,Monument / Landmark


In [20]:
#Changing venue categories into numerical values for analysis using one-hot encoding

roh = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
roh['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [roh.columns[-1]] + list(roh.columns[:-1])
roh = roh[fixed_columns]
print(roh.shape)
roh.head()

(3400, 154)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Café,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Shop,Donut Shop,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Hill,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Night Market,Nightclub,Noodle House,Optical Shop,Paper / Office Supplies Store,Park,Perfume Shop,Pet Store,Pharmacy,Pizza Place,Playground,Poke Place,Pool,Print Shop,Pub,Residential Building (Apartment / Condo),Resort,Restaurant,Salon / Barbershop,Sandwich Place,Satay Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Lodge,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track Stadium,Udon Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store
0,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Afif,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
rgrp = roh.groupby(["Neighborhoods"]).mean().reset_index()
print(rgrp.shape)
rgrp.head()

(34, 154)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Café,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Shop,Donut Shop,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Hill,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Night Market,Nightclub,Noodle House,Optical Shop,Paper / Office Supplies Store,Park,Perfume Shop,Pet Store,Pharmacy,Pizza Place,Playground,Poke Place,Pool,Print Shop,Pub,Residential Building (Apartment / Condo),Resort,Restaurant,Salon / Barbershop,Sandwich Place,Satay Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Lodge,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track Stadium,Udon Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store
0,'Uyayna,0.00,0.01,0.0,0.0,0.01,0.0,0.01,0.01,0.00,0.03,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.05,0.01,0.0,0.01,0.00,0.03,0.0,0.01,0.00,0.01,0.01,0.01,0.01,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.00,0.00,0.01,0.0,0.01,0.0,0.02,0.12,0.02,0.01,0.00,0.0,0.01,0.00,0.04,0.0,0.01,0.00,0.0,0.01,0.0,0.01,0.0,0.0,0.02,0.00,0.00,0.02,0.04,0.0,0.00,0.00,0.01,0.01,0.03,0.01,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.00,0.02,0.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.00,0.0,0.01,0.02,0.05,0.00,0.01,0.01,0.0,0.0,0.0,0.00,0.01,0.00,0.0,0.01,0.01,0.00,0.0,0.02,0.0,0.0,0.01,0.0,0.00,0.0,0.00,0.0,0.0
1,Afif,0.00,0.01,0.0,0.0,0.01,0.0,0.01,0.01,0.00,0.03,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.05,0.01,0.0,0.01,0.00,0.03,0.0,0.01,0.00,0.01,0.01,0.01,0.01,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.00,0.00,0.01,0.0,0.01,0.0,0.02,0.12,0.02,0.01,0.00,0.0,0.01,0.00,0.04,0.0,0.01,0.00,0.0,0.01,0.0,0.01,0.0,0.0,0.02,0.00,0.00,0.02,0.04,0.0,0.00,0.00,0.01,0.01,0.03,0.01,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.00,0.02,0.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.00,0.0,0.01,0.02,0.05,0.00,0.01,0.01,0.0,0.0,0.0,0.00,0.01,0.00,0.0,0.01,0.01,0.00,0.0,0.02,0.0,0.0,0.01,0.0,0.00,0.0,0.00,0.0,0.0
2,Al Artawiyah,0.00,0.01,0.0,0.0,0.01,0.0,0.01,0.01,0.00,0.03,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.05,0.01,0.0,0.01,0.00,0.03,0.0,0.01,0.00,0.01,0.01,0.01,0.01,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.00,0.00,0.01,0.0,0.01,0.0,0.02,0.12,0.02,0.01,0.00,0.0,0.01,0.00,0.04,0.0,0.01,0.00,0.0,0.01,0.0,0.01,0.0,0.0,0.02,0.00,0.00,0.02,0.04,0.0,0.00,0.00,0.01,0

In [22]:
rmall = rgrp[["Neighborhoods","Shopping Mall"]]
rmall.head()

,Neighborhoods,Shopping Mall
0,'Uyayna,0.00
1,Afif,0.00
2,Al Artawiyah,0.00
3,Al Bir,0.00
4,Al Majma'ah,0.02


In [24]:
K = 3

rclstr = rmall.drop(["Neighborhoods"], axis = 1)
kmeans = KMeans(n_clusters = K, random_state = 0).fit(rclstr)

In [27]:
rmrg = rmall.copy()
rmrg["Cluster Labels"] = kmeans.labels_
rmrg.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
rmrg.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,'Uyayna,0.00,0
1,Afif,0.00,0
2,Al Artawiyah,0.00,0
3,Al Bir,0.00,0
4,Al Majma'ah,0.02,1


In [28]:
rmrg = rmrg.join(rdf.set_index("Neighborhood"), on="Neighborhood")
print(rmrg.shape)
rmrg.head()

(34, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,'Uyayna,0.00,0,3.14789,101.69405
1,Afif,0.00,0,3.14789,101.69405
2,Al Artawiyah,0.00,0,3.14789,101.69405
3,Al Bir,0.00,0,3.14789,101.69405
4,Al Majma'ah,0.02,1,3.14327,101.71070


In [29]:
#Sorting by clusters
rmrg.sort_values(["Cluster Labels"], inplace=True)
rmrg

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,'Uyayna,0.00,0,3.14789,101.69405
31,Ushaiger,0.00,0,3.14789,101.69405
30,Tharmada'a,0.00,0,3.14789,101.69405
29,Thadig,0.00,0,3.14789,101.69405
28,Sudair Industrial City,0.00,0,3.19101,101.70778
27,Shaqraa,0.00,0,3.14789,101.69405
25,Manfuha,0.00,0,3.14789,101.69405
24,Layla (town),0.00,0,3.14789,101.69405
23,Jalajil,0.00,0,3.14789,101.69405
22,H̨awţah Sudayr,0.00,0,3.14789,101.69405


In [30]:
mapfinal = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(K)
ys = [i+x+(i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(rmrg['Latitude'], rmrg['Longitude'], rmrg['Neighborhood'], rmrg['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapfinal)
       
mapfinal

In [31]:
#cluster 0
rmrg.loc[rmrg['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,'Uyayna,0.0,0,3.14789,101.69405
31,Ushaiger,0.0,0,3.14789,101.69405
30,Tharmada'a,0.0,0,3.14789,101.69405
29,Thadig,0.0,0,3.14789,101.69405
28,Sudair Industrial City,0.0,0,3.19101,101.70778
27,Shaqraa,0.0,0,3.14789,101.69405
25,Manfuha,0.0,0,3.14789,101.69405
24,Layla (town),0.0,0,3.14789,101.69405
23,Jalajil,0.0,0,3.14789,101.69405
22,H̨awţah Sudayr,0.0,0,3.14789,101.69405


In [32]:
#cluster 1
rmrg.loc[rmrg['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Al Majma'ah,0.02,1,3.14327,101.71070
13,Arwa (village),0.02,1,3.11092,101.66187


In [33]:
#cluster 2
rmrg.loc[rmrg['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
26,Riyadh,0.01,2,3.17866,101.74142


As is shown above, we first notice that there are not a lot of malls in the capital city of Riyadh. The concentration of malls lies within cluster 1, with the availabilit